In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from keras import optimizers
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from keras import layers 
from keras import models
import cv2 # OpenCV module that handles computer vision methods


In [3]:
# libraries for a CNN
import tensorflow as tf
from tensorflow import keras


from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense

In [4]:
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
import imageio
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model

In [5]:
import os, shutil

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
from keras.applications import Xception

conv_base = Xception(weights='imagenet',
                  include_top=False,
                  input_shape=(192, 192, 3))

conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 192, 192, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 95, 95, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 95, 95, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 95, 95, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [8]:
image_path = '/content/drive/My Drive/CSC 375 Final Project/30_Classes_Separated_Data/'
train_dir = '/content/drive/My Drive/CSC 375 Final Project/30_Classes_Separated_Data/train/'
test_dir = '/content/drive/My Drive/CSC 375 Final Project/30_Classes_Separated_Data/test/'
validation_dir = '/content/drive/My Drive/CSC 375 Final Project/30_Classes_Separated_Data/validation/'

num_of_categories = 30
image_size = 192
batch_size = 16

In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = keras.preprocessing.image.ImageDataGenerator(
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      #zca_whitening=True, 
      #shear_range=0.2,
      #brightness_range=[0.9,1.1], 
      zoom_range=0.1,
      preprocessing_function=None, dtype='float32')

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)


train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size), 
        color_mode='rgb', 
        classes=None, 
        interpolation='hamming',
        class_mode='categorical', 
        batch_size=batch_size,
        shuffle=True)

validation_generator = test_datagen.flow_from_directory(
        validation_dir, 
        target_size=(image_size, image_size), 
        color_mode='rgb', 
        classes=None, 
        interpolation='hamming',
        class_mode='categorical', 
        batch_size=batch_size, 
        shuffle=False)

Found 3640 images belonging to 30 classes.
Found 796 images belonging to 30 classes.


In [10]:
conv_base.trainable = False;
conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 192, 192, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 95, 95, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 95, 95, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 95, 95, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
for layer in conv_base.layers[:249]:
    layer.trainable = False
for layer in conv_base.layers[120:]:
    layer.trainable = True
  
conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 192, 192, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 95, 95, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 95, 95, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 95, 95, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [14]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu', input_dim=6 * 6 * 2048))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3)) 
model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5)) 
model.add(layers.Dense(30, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 6, 6, 2048)        20861480  
_________________________________________________________________
flatten_3 (Flatten)          (None, 73728)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                4718656   
_________________________________________________________________
batch_normalization_14 (Batc (None, 64)                256       
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
batch_normalization_15 (Batc (None, 64)               

In [ ]:
from keras import optimizers

model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

history = model.fit( train_generator,
                        steps_per_epoch = 3640//batch_size,
                        epochs = 50,
                        validation_data = validation_generator,
                        validation_steps = 796//batch_size,
                        verbose = 2)

with open('/content/drive/My Drive/CSC 375 Final Project/xception_trainHistoryDict_v6.pickle', 'wb') as file_pi:
  pickle.dump(history.history, file_pi)

tic = time.process_time()
model_ft.save('/content/drive/My Drive/CSC 375 Final Project/xception_trainHistoryDict_v6.h5')
toc = time.process_time()
print("Processing time: ", 1000*(toc-tic), "ms")

Epoch 1/50


KeyboardInterrupt: ignored